In [1]:
from rlcube.models.models import DNN
from rlcube.envs.cube2 import Cube2Env
import torch

net = DNN()
net.load("models/model_best.pth")
net.eval()

DNN(
  (fc_in): Linear(in_features=144, out_features=512, bias=True)
  (residual_blocks): ModuleList(
    (0-3): 4 x ResidualBlock(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=512, out_features=1024, bias=True)
      (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc2): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
  (fc_value): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc_policy): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=12, bias=True)
  )
)

In [ ]:
import numpy as np

env = Cube2Env()

actions = []
obs = []
for _ in range(10):
    action = env.action_space.sample()
    actions.append(action.item())
    env.step(action)
    obs.append(env.obs())

obs = torch.tensor(np.array(obs), dtype=torch.float32)
values, policies = net(obs)
print(actions)
print(values)

from rlcube.models.search import MonteCarloTree

tree = MonteCarloTree(env.obs(), max_simulations=1000)
if tree.is_solved:
    print([action for _, action in tree.solved_path])

[11, 11, 3, 10, 9, 4, 5, 3, 11, 11]
tensor([[ 1.2608],
        [ 0.2146],
        [-0.8424],
        [-0.6595],
        [-0.4404],
        [-1.2381],
        [-0.4404],
        [-1.6949],
        [-3.1237],
        [-2.8188]], grad_fn=<AddmmBackward0>)


  9%|▉         | 469/5000 [00:04<00:48, 94.14it/s] 


KeyboardInterrupt: 

In [6]:
tree.root.N

defaultdict(<function rlcube.models.search.Node.__init__.<locals>.<lambda>()>,
            {0: 400,
             1: 0,
             2: 0,
             3: 0,
             4: 0,
             5: 0,
             6: 0,
             7: 0,
             8: 0,
             9: 0,
             10: 44,
             11: 0})